In [1]:
import torch


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

'cuda'

In [2]:
usr = "char"

# Tengo GPU pero no cuda compatible, me rompe cuando intento correr con GPU
if usr == "nico":
    device="cpu"
    num_workers = 4
    
if usr == "char":
    num_workers = 0

In [3]:
from catdog.dataset import CatDogDataset

In [4]:
DATA_DIR = '../data/'

In [5]:
import pandas as pd


train_set_pd = pd.read_csv(DATA_DIR + 'train.csv')
train_set_torch = CatDogDataset(train_set_pd)
print(f"Loaded train set with length {len(train_set_torch)}")

test_set_pd = pd.read_csv(DATA_DIR + 'test.csv')
test_set_torch = CatDogDataset(test_set_pd)
print(f"Loaded test set with length {len(test_set_torch)}")

Loaded train set with length 2948
Loaded test set with length 738


In [6]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set_torch, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set_torch, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [7]:
import numpy as np


seed = 142
torch.manual_seed(seed)
np.random.seed(seed)

#### test image

import numpy as np
from PIL import Image


test_img_path = DATA_DIR + 'images/' + cat_dog_df["file"].iloc[0]
test_img = Image.open(test_img_path).convert("RGB")

In [8]:
from catdog.models.feedforward import MLPClassifier

model_kwargs = {'input_size': 500*500*3, 'hidden_sizes': (50, 10)}
model = MLPClassifier(**model_kwargs)
model.to(device)

/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


MLPClassifier(
  (AUROC): AUROC()
  (Precision): Precision()
  (Recall): Recall()
  (model): Sequential(
    (0): Linear(in_features=750000, out_features=50, bias=True)
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=50, out_features=10, bias=True)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): CatDogOutput(
      (classify_layer): Linear(in_features=10, out_features=1, bias=True)
      (bbox_layer): Linear(in_features=10, out_features=4, bias=True)
    )
  )
)

In [9]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(142, workers=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./lightning_checkpoints/',
    filename='test_model_training',
    mode='min',
    save_top_k=3
)

logger = TensorBoardLogger("tb_logs", name="MLP_classifier")

trainer = Trainer(gpus=int(device=='cuda'), # hacky way to say 0 or 1 
                  max_epochs=1,
                  logger=logger,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=15, mode='min'),
                             checkpoint_callback])

Global seed set to 142
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [10]:
### Train and eval model

trainer.fit(model, train_loader, test_loader)  # here we are using test set as validation set
trainer.test(ckpt_path="best", dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509

  | Name      | Type       | Params
-----------------------------------------
0 | AUROC     | AUROC      | 0     
1 | Precision | Precision  | 0     
2 | Recall    | Recall     | 0     
3 | model     | Sequential | 37.5 M
-----------------------------------------
37.5 M    Trainable params
0         Non-trainable params
37.5 M    Total params
150.003   Total estimated model params size (MB)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exis

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
Global seed set to 142
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) i

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Restoring states from the checkpoint path at /alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints/test_model_training-v3.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints/test_model_training-v3.ckpt
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_auroc': 0.027530431747436523,
 'test_bbox_loss': 0.021727534011006355,
 'test_classification_loss': 0.631763219833374,
 'test_loss': 0.6483300055066744,
 'test_precision': 0.0,
 'test_recall': 0.0}
--------------------------------------------------------------------------------


[{'test_precision': 0.0,
  'test_recall': 0.0,
  'test_auroc': 0.027530431747436523,
  'test_classification_loss': 0.631763219833374,
  'test_bbox_loss': 0.021727534011006355,
  'test_loss': 0.6483300055066744}]

In [ ]:
### Eval previously trained model

checkpoint_callback.best_model_path = '/alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints/test_model_training-v8.ckpt'

best_model = model.load_from_checkpoint(checkpoint_callback.best_model_path, **model_kwargs).to(device)
trainer.test(model=best_model, dataloaders=test_loader, verbose=True)

